In [1]:
# Install required Packages
!pip install -q huggingface_hub==0.25.0 diffusers transformers accelerate ftfy torch torchvision
!pip install -q ffmpeg-python streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 82.0 MB/s eta 0:00:00


In [2]:
# Load dependencies
import os
import torch
from diffusers import StableDiffusionPipeline
import ffmpeg
from PIL import Image

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
# Create working folders
os.makedirs("data/generated_videos/temp", exist_ok=True)

In [4]:
# Load the Pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    revision="fp16" if torch.cuda.is_available() else None,
    low_cpu_mem_usage = True
).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
# generate frames
def generate_sequence(prompt: str, num_frames: str, int = 8, output_dir: str = "data/generated_videos/temp"):
  images = []
  for i in range (num_frames):
    image = pipe(f"{prompt}, frame {i + 1}", guidance_scale=7.5).images[0]
    path = os.path.join(output_dir, f"frame_{i:03}.png")
    image.save(path)
    images.append(path)
  return images

In [8]:
# Create a video from frames
def create_video(frames_dir="data/generated_videos/temp", output_path="output.mp4", fps=2):
  (
      ffmpeg
      .input(os.path.join(frames_dir, 'frame_%03d.png'), framerate=fps)
      .output(output_path, vcodec='libx264', pix_fmt='yuv420p')
      .overwrite_output()
      .run()
  )

  return output_path

In [9]:
# Run a sample prompt
frames = generate_sequence("a fire on a factory floor", num_frames=6)
video_path = create_video()

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [11]:
from IPython.display import Video
Video(video_path)